# **Ajuste de Hiperparámetros** ⚙️🤖

Bienvenidos a esta nueva aventura llena de aprendizajes 🚀. Como el título lo anuncia, estaremos explorando el fascinante mundo de los **hiperparámetros** y cómo ajustarlos adecuadamente. Pero antes de sumergirnos en este viaje, es crucial aclarar una serie de conceptos para que vayas comprendiendo el paso a paso que aquí estaremos desarrollando 🧭.

## **¿Qué son los hiperparámetros?** ❓

Esta, sin duda, debe ser la pregunta más importante que te estés planteando en este momento 🤔. Pues bien, los hiperparámetros son las configuraciones empleadas para estructurar modelos de *Machine Learning*. Podríamos decir que son ajustes por defecto que nosotros mismos aplicamos antes de dar inicio al proceso de entrenamiento; esto nos ayuda a controlar aspectos tanto del entrenamiento como de la estructura del modelo ⚒️.

El *fine tuning* de estos hiperparámetros será el enfoque principal de este minicurso 🧠, de tal manera que seamos capaces de ajustarlos sistemáticamente, lo que posteriormente permitirá optimizar el rendimiento de nuestros modelos 📈.

### Algunos datos importantes sobre el *fine tuning* de los hiperparámetros 🛠️:

1. Los ajustes por defecto de los hiperparámetros a veces no se adaptan completamente a las características de nuestras bases de datos y sus requerimientos 📊; por eso es necesario intervenir manualmente y establecer nuevos parámetros "por defecto" ✍️.
2. Un ajuste adecuado puede incrementar de manera significativa la **precisión**, **eficiencia** y, en términos generales, el **rendimiento** del modelo 💡.
3. Ayuda a alcanzar un equilibrio entre el **sobreajuste** (*overfitting*) y el **subajuste** (*underfitting*) ⚖️, llevando al modelo a una mejor generalización frente a datos no vistos 🔍.
4. Todos los modelos responden de forma diferente a los distintos tipos de ajustes 🧪, por lo que es necesario realizar este proceso en cada caso específico (es muy difícil reutilizar el proceso como en los *pipelines*) 🔄.


Como siempre, nuestros primeros pasos están orientados a la importación de nuestras bases de datos 📂 y, posteriormente, al estudio de la base de datos seleccionada para este ejercicio.

In [103]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import (
    train_test_split, cross_val_score,
    GridSearchCV, RandomizedSearchCV,
    RepeatedKFold
)
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import root_mean_squared_error

## **🌌 Nuestra Base de Datos**

Como es costumbre, en los recursos de este minicurso te dejaré la base de datos que voy a estar empleando 📊. En caso de que quieras hacerlo con una base diferente, no hay ningún inconveniente. Sin embargo, si es tu primera vez haciendo ajuste de hiperparámetros, te recomiendo usar una base de datos limpia 🧼 y cuyas variables sean exclusivamente numéricas 🔢.

En esta oportunidad estaré usando la base de datos **Stellar Classification Dataset - SDSS17** ✨, extraída del recurso *Kaggle*. Un poco de contexto sobre esta base de datos:

En astronomía 🌠, la clasificación estelar consiste en clasificar las estrellas según sus características espectrales 🔭. El sistema de clasificación de galaxias 🌌, cuásares 🌟 y estrellas ⭐ es uno de los más fundamentales en astronomía. La catalogación temprana de las estrellas y su distribución en el cielo permitió comprender que conforman nuestra galaxia 💫. Tras la distinción de que Andrómeda era una galaxia independiente de la nuestra, se comenzaron a estudiar numerosas galaxias a medida que se construían telescopios más potentes 🛰️. Este conjunto de datos tiene como objetivo clasificar estrellas, galaxias y cuásares según sus características espectrales 📈.

Esta base de datos se compone por **100.000 observaciones del espacio** 🚀, tomadas del *Sloan Digital Sky Survey*. Cada una de las observaciones se encuentra descrita por **17 variables** y una clase que indica si se trata de una estrella, galaxia o cuásar. Las variables son:

- `obj_ID`: 🆔 Identificador único del objeto en el catálogo de imágenes.
- `alpha`: 🧭 Ángulo de ascensión recta (época J2000).
- `delta`: 🧭 Ángulo de declinación (época J2000).
- `u`: 🌈 Filtro ultravioleta.
- `g`: 💚 Filtro verde.
- `r`: ❤️ Filtro rojo.
- `i`: 💡 Filtro infrarrojo cercano.
- `z`: 🌙 Filtro infrarrojo.
- `run_ID`: 🔁 Número de ejecución.
- `rerun_ID`: 🔁 Número de reejecución.
- `cam_col`: 📷 Columna de cámara.
- `field_ID`: 🗺️ Número de campo.
- `spec_obj_ID`: 🧬 ID único para objetos espectroscópicos.
- `class`: 🏷️ Clase del objeto (galaxia, estrella o cuásar).
- `redshift`: 🚦 Valor de corrimiento al rojo.
- `plate`: 💿 ID de la placa.
- `MJD`: 🗓️ Fecha Juliana Modificada.
- `fiber_ID`: 🔌 ID de la fibra óptica usada en la observación.

In [104]:
df=pd.read_csv(r".\star_classification.csv", delimiter=",", quotechar='"')
df

,obj_ID,alpha,delta,u,g,r,i,z,run_ID,rerun_ID,cam_col,field_ID,spec_obj_ID,class,redshift,plate,MJD,fiber_ID
0,1.237661e+18,135.689107,32.494632,23.87882,22.27530,20.39501,19.16573,18.79371,3606,301,2,79,6.543777e+18,GALAXY,0.634794,5812,56354,171
1,1.237665e+18,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,4518,301,5,119,1.176014e+19,GALAXY,0.779136,10445,58158,427
2,1.237661e+18,142.188790,35.582444,25.26307,22.66389,20.60976,19.34857,18.94827,3606,301,2,120,5.152200e+18,GALAXY,0.644195,4576,55592,299
3,1.237663e+18,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.25010,4192,301,3,214,1.030107e+19,GALAXY,0.932346,9149,58039,775
4,1.237680e+18,345.282593,21.183866,19.43718,17.58028,16.49747,15.97711,15.54461,8102,301,3,137,6.891865e+18,GALAXY,0.116123,6121,56187,842
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,1.237679e+18,39.620709,-2.594074,22.16759,22.97586,21.90404,21.30548,20.73569,7778,301,2,581,1.055431e+19,GALAXY,0.000000,9374,57749,438
99996,1.237679e+18,29.493819,19.798874,22.69118,22.38628,20.45003,19.75759,19.41526,7917,301,1,289,8.586351e+18,GALAXY,0.404895,7626,56934,866
99997,1.237668e+18,224.587407,15.700707,21.16916,19.26997,18.20428,17.69034,17.35221,5314,301,4,308,3.112008e+18,GALAXY,0.143366,2764,54535,74
99998,1.237661e+18,212.268621,46.660365,25.35039,21.63757,19.91386,19.07254,18.62482,3650,301,4,131,7.601080e+18,GALAXY,0.455040,6751,56368,470


In [105]:
df['class'].value_counts()

class
GALAXY    59445
STAR      21594
QSO       18961
Name: count, dtype: int64

## 🛠️ Preparación de la Base de Datos

Antes de continuar, vamos a realizar algunos cambios en nuestra base de datos 🔧. 

1. En primer lugar, eliminaremos algunas variables que probablemente pueden generar ruido o provocar fuga de datos 🧹.
2. En segundo lugar, tomaremos como **variable objetivo** `class` 🏷️. Sin embargo, como esta es una variable categórica, será necesario transformarla a un formato numérico 🔢 para evitar problemas futuros en el proceso de modelado.
3. Finalmente, realizaremos un pequeño **Análisis Exploratorio de Datos (EDA)** 🔍 para comprender mejor el comportamiento general de nuestra base de datos y así tomar decisiones más informadas durante el ajuste de hiperparámetros.

¡Manos a la obra! 🚀


In [106]:
# Eliminación de variables
df = df.drop(['obj_ID', 'spec_obj_ID', 'plate', 
              'fiber_ID', 'run_ID', 
              'rerun_ID','cam_col','field_ID', 'MJD'], axis=1)

In [107]:
# Transformacion de class
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['class'] = le.fit_transform(df['class'])
df.head()

# Donde 0 es "GALAXY", 2 es "STAR", 1 es "QSO"

#Definimos la variable objetivo Y e las variables predictoras X:
y=df["class"]
X=df.drop(['class'], axis=1) #debemos eliminar class del resto del conjnto de datos
X.head()

,alpha,delta,u,g,r,i,z,redshift
0,135.689107,32.494632,23.87882,22.27530,20.39501,19.16573,18.79371,0.634794
1,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,0.779136
2,142.188790,35.582444,25.26307,22.66389,20.60976,19.34857,18.94827,0.644195
3,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.25010,0.932346
4,345.282593,21.183866,19.43718,17.58028,16.49747,15.97711,15.54461,0.116123


In [108]:
# EDA
X.isna().sum().reset_index().rename(columns={"index":"Variable", 0:"Valores faltantes"})

,Variable,Valores faltantes
0,alpha,0
1,delta,0
2,u,0
3,g,0
4,r,0
5,i,0
6,z,0
7,redshift,0


Como podemos observar nuestra base se encuentra limpia, no tiene datos faltantes. Ahora echemos un vistazo a las estadísticas descriptivas.

In [109]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
alpha,100000.0,177.629117,96.502241,0.005528,127.518222,180.900700,233.895005,359.999810
delta,100000.0,24.135305,19.644665,-18.785328,5.146771,23.645922,39.901550,83.000519
u,100000.0,21.980468,31.769291,-9999.000000,20.352353,22.179135,23.687440,32.781390
g,100000.0,20.531387,31.750292,-9999.000000,18.965230,21.099835,22.123767,31.602240
r,100000.0,19.645762,1.854760,9.822070,18.135828,20.125290,21.044785,29.571860
i,100000.0,19.084854,1.757895,9.469903,17.732285,19.405145,20.396495,32.141470
z,100000.0,18.668810,31.728152,-9999.000000,17.460677,19.004595,19.921120,29.383740
class,100000.0,0.621490,0.816778,0.000000,0.000000,0.000000,1.000000,2.000000
redshift,100000.0,0.576661,0.730707,-0.009971,0.054517,0.424173,0.704154,7.011245


> ⚠️ Ya que el propósito de este minicurso no es ahondar en los *insights* que nos arroja el EDA, no estaremos analizando sus resultados en detalle. 

Sin embargo, es **muy importante** que revises con detenimiento la información obtenida 🔍, ya que esta puede resultar de gran ayuda más adelante, especialmente durante la selección de variables y el ajuste de hiperparámetros 🧠📊.


## 🔭 Partición de los datos

Una vez hemos estudiado nuestra base de datos, vamos a proceder con la **partición de los datos**, pues como recordarás de nuestro curso de aprendizaje supervisado, lo usual es dividir nuestra data en dos conjuntos: un **80% para entrenar el modelo** y un **20% para evaluar su desempeño**.

🔁 Recuerda que estos porcentajes (80-20) son únicamente una guía, y puedes ajustarlos según las dimensiones y características de tus datos.




In [110]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,      
    random_state=123     
)

Ya que tenemos la partición de nuestro datos podemos proceder con la ejecución del primer modelo un Baseline con cross validation.

## ✨ Baseline model

Es importante que comprendas que existen diferentes tipos de modelos de línea base, principalmente de **clasificación** y **regresión**. Dada la naturaleza de nuestra variable objetivo, emplearemos un modelo de clasificación.

Este tipo de modelos cumplen un doble propósito:  
1. Proporcionan un punto de referencia para medir la eficiencia de modelos más complejos.  
2. Ofrecen una línea base de desempeño contra la cual se pueden comparar los avances obtenidos en el entrenamiento. 😊


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

# Modelo de linea base
baseline_model = DecisionTreeClassifier(random_state=42)

# Validación cruzada
cv_scores = cross_val_score(
    baseline_model,
    X_train, y_train,
    cv=5,
    scoring='roc_auc_ovr'  #estaremos usando el ROC, ya que nuestros datos no se encuentran balanceados
)

# Resultados
baseline_avg_roc = cv_scores.mean()
baseline_std_roc = cv_scores.std()

In [116]:
# Comparación de los resultados 
df_performance_comp = pd.DataFrame(columns=["Model", "Avg_ROC", "Std_ROC"])
df_performance_comp.loc[0, :] = ["Baseline", baseline_avg_roc, baseline_std_roc]
df_performance_comp

,Model,Avg_ROC,Std_ROC
0,Baseline,0.968542,0.001294


 📊 **¿Qué quieren decir estos resultados?**

✨ ¡En términos generales vamos súper bien! Tener un **ROC promedio de 96.85% (0.9685)** ✅ nos indica que nuestro modelo acertó en ese porcentaje mientras hacía las predicciones de validación cruzada 🔁. Es decir, ¡el margen de error fue muy pequeño! 🧠📉

📏 Por su parte, tener una desviación estándar tan bajita (**0.0012**) nos sugiere que el modelo es **relativamente estable y consistente** 🧘‍♂️📊. Esto se traduce como: **nuestro modelo funcionará bien sin importar cómo dividamos los datos** 💪✨.


## 🌲 Red de Búsqueda para el Ajuste de Hiperparámetros

En esta etapa ejecutaremos un modelo de **árbol de decisión** 🌳 utilizando **búsqueda en malla (Grid Search)** 🧩 con **validación cruzada** 🔁. El objetivo es identificar los **hiperparámetros óptimos** 🛠️ para mejorar el rendimiento del modelo de clasificación que entrenamos previamente 🧠📈.


In [117]:
# Red de parámetros
red_param = {
    'max_depth': [4, 5, 7, 8], #profundidad máxima del árbol
    'min_samples_split': [5, 10, 15], #número minimo de muestras para dividir un nodo
    'min_samples_leaf': [5, 10, 15] #número minimo de muestras por hojas
}

# Initialize Grid Search with Cross-Validation
busqueda_red = GridSearchCV(
    DecisionTreeClassifier(random_state=123),
    param_grid=red_param,
    cv=5, # realiza 5 particiones de validación cruzada
   
    scoring='roc_auc_ovr',
    n_jobs=-1, # Emplea todos los núcleso del procesador para hacer el proceso más rápido
    verbose=1, # imprime el progreso
    refit=True # cuando encuentra la mejor combinacion, vuelve a estimar todo el modelo
)

# Fit Grid Search
busqueda_red.fit(X_train, y_train)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=123),
             n_jobs=-1,
             param_grid={'max_depth': [4, 5, 7, 8],
                         'min_samples_leaf': [5, 10, 15],
                         'min_samples_split': [5, 10, 15]},
             scoring='roc_auc_ovr', verbose=1)

Si bien nuestro modelo ya fue ejecutado, no tenemos los resultados a simple vista así que para poderlos explorar y ver detalladamente a que conslusión se llegó ejecutareos el siguiente código:

In [118]:
# Exploring the results
resultados_malla = pd.DataFrame(busqueda_red.cv_results_)
resultados_malla = resultados_malla.drop(
    columns=[
        "mean_fit_time", "std_fit_time", "mean_score_time",
        "std_score_time", "params"
    ]
)
resultados_malla.sort_values(by="rank_test_score", ascending=True)

,param_max_depth,param_min_samples_leaf,param_min_samples_split,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
34,8,15,10,0.991355,0.992871,0.993250,0.991178,0.991615,0.992054,0.000842,1
35,8,15,15,0.991355,0.992871,0.993250,0.991178,0.991615,0.992054,0.000842,1
33,8,15,5,0.991355,0.992871,0.993250,0.991178,0.991615,0.992054,0.000842,1
23,7,10,15,0.992320,0.992697,0.992793,0.990498,0.991695,0.992001,0.000844,4
22,7,10,10,0.992320,0.992697,0.992793,0.990498,0.991695,0.992001,0.000844,4
21,7,10,5,0.992320,0.992697,0.992793,0.990498,0.991695,0.992001,0.000844,4
26,7,15,15,0.992372,0.992752,0.993042,0.990348,0.991414,0.991986,0.000986,7
24,7,15,5,0.992372,0.992752,0.993042,0.990348,0.991414,0.991986,0.000986,7
25,7,15,10,0.992372,0.992752,0.993042,0.990348,0.991414,0.991986,0.000986,7
20,7,5,15,0.992199,0.992553,0.992818,0.990217,0.991693,0.991896,0.000920,10


📊✨ En este *data frame* podemos ver cada uno de los **modelos que se ejecutaron**, con sus correspondientes características: la **profundidad máxima alcanzada** 🌳, la **cantidad mínima de hojas por muestra** 🍃, la **cantidad mínima de particiones** ✂️, el **nivel de ROC** obtenido en cada intento 🎯, entre otros.

✅ Ahora, con base en esta información, vamos a solicitar que nos arroje **cuál es la mejor combinación**, es decir, **aquella que optimiza nuestro modelo** 🚀.


In [119]:
# ¿Cuál es la mejor combinación?
mejor_combinacion = busqueda_red.best_params_
print(f'La búsquea en malla encuentra que la mejor combinación de parámetros es: {mejor_combinacion}')

# ¿Cual es el mejor estimador?
mejor_modelo = busqueda_red.best_estimator_
print(f'Se determina que el mejor estimador es:{mejor_modelo}')

La búsquea en malla encuentra que la mejor combinación de parámetros es: {'max_depth': 8, 'min_samples_leaf': 15, 'min_samples_split': 5}
Se determina que el mejor estimador es:DecisionTreeClassifier(max_depth=8, min_samples_leaf=15, min_samples_split=5,
                       random_state=123)


✅ **Se concluye que la mejor combinación de parámetros fue aquella con:**

- 🌳 **Profundidad máxima:** 8  
- 🍃 **Cantidad mínima de hojas por muestra:** 15  
- ✂️ **Cantidad mínima de particiones por muestra:** 5 


### 📊 Comparación de los dos modelos que tenemos

En este momento vamos a hacer algo súper importante 🤓, y es comparar las métricas entre los dos modelos que tenemos hasta el momento: el de línea base (**Baseline model**) y el optimizado que acabamos de encontrar con ayuda de la búsqueda tipo malla 🔍.

Estaremos comparando las métricas del **promedio del ROC** y su **desviación estándar**, y veremos qué tanto varían de uno a otro 📈.


In [121]:
# Evaluacon del modelo de búsqueda tipo malla con validación cruzada
cv_valores_ajustados = cross_val_score(
    mejor_modelo, X_train, y_train, cv=5,
    scoring='roc_auc_ovr'
)

avg_ajustado_roc = cv_valores_ajustados.mean()
std_ajustado_roc = cv_valores_ajustados.std()

In [122]:
# Comparación
df_performance_comp.loc[1, :] = ["Búsqueda malla ajs", avg_ajustado_roc,std_ajustado_roc ]
df_performance_comp

,Model,Avg_ROC,Std_ROC
0,Baseline,0.968542,0.001294
1,Búsqueda malla ajs,0.992054,0.000842


Podemos observar cómo, efectivamente, la nueva combinación encontrada por la búsqueda tipo malla ⚙️ mejoró significativamente el rendimiento de nuestro modelo, alcanzando un **ROC de 99.20%** 📈 y una **desviación estándar aún más pequeña** 📉. Esto refleja que, con los cambios aplicados, el modelo se volvió aún más **estable** y **preciso** 💪✨.


## 🌲 Búsqueda aleatoria para el ajuste de hiperparámetro

En esta etapa ejecutaremos un modelo de árbol de decisión 🌳 utilizando búsqueda aleatoria (Random Search) 🎲 con validación cruzada 🔁. El objetivo es explorar de manera eficiente un conjunto amplio de hiperparámetros 🛠️ y encontrar aquellos que optimicen el rendimiento del modelo de regresión que entrenamos previamente 📉📈. A diferencia de la búsqueda en malla, esta técnica permite probar combinaciones al azar, lo que reduce el tiempo de cómputo sin comprometer la calidad de los resultados ⏱️⚡.

In [123]:
# Configuramos nuevamente los requerimientos que tendra nuestro árbol de desición
tree_config = {
    'max_depth': range(2, 8),
    'min_samples_split': range(5, 16),
    'min_samples_leaf': range(5, 16)
}

# Damos inicio a nuestra búsqueda aleatoria 
busqueda_aleatoria = RandomizedSearchCV(
    DecisionTreeClassifier(random_state=123),
    param_distributions=tree_config,
    n_iter=100, # combinaciones aleatorias de parámetros
    cv=5,
  
    scoring='roc_auc_ovr',
    n_jobs=-1, 
    verbose=1, 
    random_state=42
)

busqueda_aleatoria.fit(X_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


RandomizedSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=123),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': range(2, 8),
                                        'min_samples_leaf': range(5, 16),
                                        'min_samples_split': range(5, 16)},
                   random_state=42, scoring='roc_auc_ovr', verbose=1)

Si deseas visualizar cada uno de los modelos ejecutados, puedes emplear un código similar al que usamos previamente para el modelo de búsqueda en malla 🔧. 

En este caso, iremos directamente a ver **la mejor combinación** que se encontró en esta oportunidad 🥇 y la **comparación de los modelos** con los que contamos hasta el momento 📊🤖.


In [124]:
# Extraemos los mejores parámetros
busqueda_aleatoria_params = busqueda_aleatoria.best_params_
print(f'La búsqueda aleatoria indica que los mejores parámetros son: {busqueda_aleatoria_params}')

# Obtenemos el emjor estimador
busqueda_aleatoria_modelo = busqueda_aleatoria.best_estimator_
print(f'La búsqueda aleatoria indica que el mejor estimador es: {busqueda_aleatoria_modelo}')

La búsqueda aleatoria indica que los mejores parámetros son: {'min_samples_split': 13, 'min_samples_leaf': 14, 'max_depth': 7}
La búsqueda aleatoria indica que el mejor estimador es: DecisionTreeClassifier(max_depth=7, min_samples_leaf=14, min_samples_split=13,
                       random_state=123)


En esta ocasión, nuestro modelo determinó que la **combinación de parámetros que optimiza el modelo de clasificación** es la siguiente:

- 🌲 **Profundidad máxima:** 7 
- 🍃 **Cantidad mínima de hojas por muestra:** 14  
- ✂️ **Cantidad mínima de particiones por muestra:** 13  



### 📊 Comparación de los tres modelos que tenemos

En este momento vamos a hacer algo súper importante 🤓, y es comparar las métricas entre los tres modelos que tenemos hasta el momento: el de línea base (**Baseline model**), la búsqueda tipo malla 🔍 y la búsqueda aleatoria.

Estaremos comparando las métricas del **promedio del ROC** y su **desviación estándar**, y veremos qué tanto varían de uno a otro 📈.

In [125]:
# Evaluación del modelo de búsqueda a aleatoria con validación cruzada
cv_scores_busqueda_a = cross_val_score(
    busqueda_aleatoria_modelo, X_train, y_train, cv=5,
    scoring='roc_auc_ovr'
)

busqueda_a_avg_roc = cv_scores_busqueda_a.mean()
busqueda_a_std_roc = cv_scores_busqueda_a.std()

In [126]:
#Comparación actualizada

df_performance_comp.loc[2, :] = ["Búsqueda Aleatoria Ajs.", busqueda_a_avg_roc, busqueda_a_std_roc ]
df_performance_comp

,Model,Avg_ROC,Std_ROC
0,Baseline,0.968542,0.001294
1,Búsqueda malla ajs,0.992054,0.000842
2,Búsqueda Aleatoria Ajs.,0.992005,0.000944


Como podemos observar, tras ejecutar esta comparación, nos damos cuenta de que la **última metodología empleada no generó aportes significativos**. La búsqueda tipo malla 🔍 continúa presentando el **mejor nivel de ROC promedio** 📈 así como la **menor desviación estándar** 📉, lo que reafirma su solidez como técnica de optimización de hiperparámetros en este caso.


## 🛠️ Ajuste de Hiperparámetros con Optuna

Pero… ¿**qué es Optuna**? 🤔  
Es un **framework de software diseñado específicamente para la optimización eficiente y automática de hiperparámetros**. 🔍✨

### ¿Cómo funciona? 🧠

1. 🧪 **Definimos una función objetivo** con muestreo de hiperparámetros.
2. 🌲 **Emplea algoritmos eficientes como TPE**, los cuales guardan bastante similitud con los árboles de decisión.
3. ⏩ **Elimina de forma temprana** aquellos modelos o ensayos que lucen poco prometedores, lo que **aumenta la eficiencia** en el proceso de selección.


In [ ]:
# En primer lugar y lo más importante debemos instalar Optuna
#!pip install optuna

   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------- ----- 1.8/2.1 MB 8.6 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 8.1 MB/s eta 0:00:00



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [127]:
# El segundo paso será, tal y como indicamos antes, definir la fución objetivo 

def objective(intento):
    # Configuramos las caraterísticas del árbol de desición
    max_depth = intento.suggest_int('max_depth', 2, 32)
    min_samples_split = intento.suggest_int('min_samples_split', 5, 20)
    min_samples_leaf = intento.suggest_int('min_samples_leaf', 5, 20)
    

    modelo = DecisionTreeClassifier(
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        random_state=123
    )
    
    scores = cross_val_score(
        modelo, X_train, y_train, cv=5,
        scoring='roc_auc_ovr'
    )
    

    return scores.mean()
# Este segundo paso no tiene mayores dfirencias con lo que ya hemos hecho en 
# los modelos anteriores

In [128]:
# El segundo paso consiste en correr el estudio de Optuna
import optuna

analisis = optuna.create_study(direction='minimize')
analisis.optimize(
    objective, # Objective function to minimize
    n_trials=100, # Number of trials to run
    n_jobs=-1 # Use all available cores
)

[I 2025-05-09 16:06:25,203] A new study created in memory with name: no-name-2817ef75-e9c1-4b70-9ca3-f5a214334ac7


[I 2025-05-09 16:06:34,357] Trial 2 finished with value: 0.9910215595026195 and parameters: {'max_depth': 9, 'min_samples_split': 8, 'min_samples_leaf': 6}. Best is trial 2 with value: 0.9910215595026195.
[I 2025-05-09 16:06:36,588] Trial 3 finished with value: 0.9867765459297526 and parameters: {'max_depth': 13, 'min_samples_split': 9, 'min_samples_leaf': 7}. Best is trial 3 with value: 0.9867765459297526.
[I 2025-05-09 16:06:37,761] Trial 1 finished with value: 0.9896147169085037 and parameters: {'max_depth': 22, 'min_samples_split': 16, 'min_samples_leaf': 20}. Best is trial 3 with value: 0.9867765459297526.
[I 2025-05-09 16:06:37,946] Trial 0 finished with value: 0.9888792066823717 and parameters: {'max_depth': 19, 'min_samples_split': 12, 'min_samples_leaf': 17}. Best is trial 3 with value: 0.9867765459297526.
[I 2025-05-09 16:06:46,776] Trial 4 finished with value: 0.9861395761230275 and parameters: {'max_depth': 17, 'min_samples_split': 19, 'min_samples_leaf': 8}. Best is trial 

Lo que observamos arriba son todos los **estudios que Optuna ejecutó** 🧪🔁.  
Sin embargo, lo que **realmente nos interesa** es conocer cuál fue el **intento que arrojó los mejores resultados** 🏆, y **cuáles fueron las características de los parámetros** seleccionados en ese ensayo exitoso. 🧬✨

Con esta información podremos comparar el rendimiento con los modelos anteriores y evaluar si realmente hubo una mejora significativa. 📊🤓


De acuerdo con el estudio practicado por **Optuna** 🧠🔍, el mejor modelo se obtuvo en el **intento número 14** 🥇, logrando un **ROC de 95.39%** 📈.

Los **parámetros óptimos** de ese modelo fueron:

- 🌲 **Profundidad máxima:** 2  
- 🍃 **Cantidad mínima de hojas por muestra:** 5  
- ✂️ **Cantidad mínima de particiones por muestra:** 20  

Aunque este modelo tiene un desempeño ligeramente inferior al obtenido con la búsqueda tipo malla, es importante destacar la eficiencia del proceso automatizado de Optuna en la exploración del espacio de hiperparámetros. ⚙️✨

In [129]:
# Resultados del estudio
print("Mejor intento:")
print(f"-Intento número: {analisis.best_trial.number}")
print(f"-Valor de ROC: {analisis.best_value:.4f}")
print(f"-Parámetros: {analisis.best_params}")

Mejor intento:
-Intento número: 17
-Valor de ROC: 0.9539
-Parámetros: {'max_depth': 2, 'min_samples_split': 20, 'min_samples_leaf': 5}


In [144]:
# Entrenamos el modelo con los mejores parametros encontrado por Optuna
opt_mejor_modelo = DecisionTreeClassifier(**analisis.best_params, random_state=42)

### 📊 Comparación de todos los modelos que tenemos

En este momento vamos a hacer algo súper importante (como lo hemos hecho con todos los modelos previos) 🤓,  y es comparar las métricas entre los cuatro modelos que tenemos hasta el momento: el de línea base (**Baseline model**), la búsqueda tipo malla 🔍,  la búsqueda aleatoria y Optuna.

Estaremos comparando las métricas del **promedio del ROC** y su **desviación estándar**, y veremos qué tanto varían de uno a otro 📈.

In [146]:
op_valores_ajus = cross_val_score(
    opt_mejor_modelo, X_train, y_train,
    cv=5, scoring='roc_auc_ovr'
)

# Calcular el promedio y la desviación estándar de los AUC obtenidos
op_tuned_avg_roc = op_valores_ajus.mean()
op_tuned_std_roc = op_valores_ajus.std()


In [147]:
# Comparación
df_performance_comp.loc[3, :] = ["Optuna Tuned", op_tuned_avg_roc, op_tuned_std_roc]
df_performance_comp

,Model,Avg_ROC,Std_ROC
0,Baseline,0.968542,0.001294
1,Búsqueda malla ajs,0.992054,0.000842
2,Búsqueda Aleatoria Ajs.,0.992005,0.000944
3,Optuna Tuned,0.953864,0.000895


💡 **Volvemos a reiterar:** aunque a simple vista el mejor modelo podría parecer el obtenido con la **búsqueda en malla** 🔍, de acuerdo con el estudio realizado por **Optuna** 🧠, **este último resulta ser el más eficiente** en términos de recursos y tiempo de cómputo, y también **optimiza adecuadamente el rendimiento del modelo**.

✅ Esto significa que, aunque no obtuvo el mayor **ROC**, sí logró un excelente **balance entre rendimiento y eficiencia computacional**, lo cual es fundamental en proyectos reales donde el tiempo y los recursos son limitados. ⚙️⏱️


## 🚀 ¡Fase final!

Lo que hemos visto hasta el momento corresponde al desempeño de los modelos **prediciendo dentro de la misma muestra** (validación cruzada), pero **aún no se han enfrentado a los datos que no les compartimos**: ese 20% reservado para la prueba final. 🧪🔒

Vamos a verificar si el desempeño **se mantiene** al intentar predecir estos **datos ocultos**, lo cual nos dará una idea real de qué tan bien generaliza nuestro modelo. 📈🤖


In [155]:
#Modelo de linea base

from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import label_binarize


baseline_model = DecisionTreeClassifier(random_state=42)
baseline_model.fit(X_train, y_train)

# ROC AUC (multiclase)
baseline_test_roc = roc_auc_score(
    label_binarize(y_test, classes=baseline_model.classes_),
    baseline_model.predict_proba(X_test),
    average='macro',
    multi_class='ovr'
)

print(f'Linea Base Test ROC AUC: {baseline_test_roc:.4f}')

Linea Base Test ROC AUC: 0.9694


In [156]:
# Modelo busqueda tipo malla
gs_test_roc = roc_auc_score(
    label_binarize(y_test, classes=mejor_modelo.classes_),
    mejor_modelo.predict_proba(X_test),
    average='macro',
    multi_class='ovr'
)

print(f'Mejor modelo búsqueda malla Test ROC AUC: {gs_test_roc:.4f}')

Mejor modelo búsqueda malla Test ROC AUC: 0.9921


In [157]:
rs_test_roc = roc_auc_score(
    label_binarize(y_test, classes=busqueda_aleatoria_modelo.classes_),
    busqueda_aleatoria_modelo.predict_proba(X_test),
    average='macro',
    multi_class='ovr'
)

print(f'Mejor modelo busqueda aleatoria Test ROC AUC: {gs_test_roc:.4f}')

Mejor modelo busqueda aleatoria Test ROC AUC: 0.9921


In [158]:

opt_mejor_modelo.fit(X_train, y_train)

op_test_roc = roc_auc_score(
    label_binarize(y_test, classes=opt_mejor_modelo.classes_),
    opt_mejor_modelo.predict_proba(X_test),
    average='macro',
    multi_class='ovr'
)

print(f'Modelo Optuna Test ROC AUC: {op_test_roc:.4f}')


Modelo Optuna Test ROC AUC: 0.9563


## 🎓 **Conclusiones**

Durante este entretenido mini curso nos enfocamos en un tema clave para mejorar modelos de machine learning: **el ajuste de hiperparámetros** 🔧🤖. Exploramos diferentes metodologías que permiten optimizar el rendimiento de nuestros modelos, comparando sus resultados y entendiendo en qué casos cada una brilla más.

Aprendimos a implementar y analizar:

- **Modelos de línea base**, que nos sirven como punto de referencia inicial 📏.
- **Búsqueda en malla (Grid Search)**, que explora combinaciones exhaustivas de hiperparámetros 🧩.
- **Optuna**, un framework moderno que utiliza estrategias inteligentes como el algoritmo TPE y la parada temprana para buscar de manera más eficiente 🌲✨.

Uno de los aprendizajes más valiosos fue entender que **no siempre la tecnica con el ROC más alto es el mejor** ⚖️. En ocasiones, como vimos con **Optuna**, una tecnica que logra un buen balance entre **precisión, estabilidad y eficiencia computacional** puede ser más recomendable.


✨ **¡Espero que hayas disfrutado mucho este recorrido!**  
Y recuerda: el ajuste de hiperparámetros no es solo una tarea técnica, ¡también es una oportunidad para conocer mejor tu modelo y sacarle el máximo provecho! 🚀🔍

Sigue practicando y no dudes en experimentar con nuevos algoritmos y técnicas. ¡El mundo del aprendizaje automático siempre tiene algo nuevo por descubrir! 🌌💡
